In [ ]:
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
from bs4 import BeautifulSoup

In [ ]:
def download(start,stop):

  active_links = []
  for indx in tqdm(range(start, stop)):        # 16000

    link = 'https://www.cricbuzz.com/profiles/' + str(indx)
    soup = BeautifulSoup(requests.get(link).content, 'lxml')

    if len(soup.find_all('table')) == 2:
      active_links.append(link)

  np.save(str(start) + '_' + str(stop) + '.npy', np.array(active_links))

  print(len(active_links), 'in', (str(start) + '_' + str(stop) + '.npy'))
  

def get_col_names(table):                                     # Get Column Names from the Table
    return [_.text.strip() for _ in table.find_all('th')]

def get_data(table):                                          # Get Table Data
    return [_.text.strip() for _ in table.find_all('td')]

def get_rows(table):                                          # Get number of rows from the Table
    return len(table.find('tbody').find_all('tr'))
    
def get_cols(table):                                          # Get number of columns from the Table
    return len(get_col_names(table))

def create_df(table):                                         # Create the DataFrame
    data   = np.reshape(get_data(table), (get_rows(table), get_cols(table)))
    return pd.DataFrame(data, columns = get_col_names(table))    

In [ ]:
download(0,30000)

100%|██████████| 5000/5000 [53:23<00:00,  1.56it/s]

423 in 25000_30000.npy


In [ ]:
links = ([_ for _ in np.load('data.npy')] + [_ for _ in np.load('data_1.npy')])

np.save('players.npy', links)

In [ ]:
final_data = []

for link in tqdm(links):

  soup = BeautifulSoup(requests.get(link).content, 'html.parser')

  try:
      details = []
      players_data = []

      # Scraping Basic Info
      details.append(soup.find('h1').text.strip())    # Name of the Player
      details.append(soup.find('h3').text.strip())    # Team Name

      basic_info = [sp.text.strip() for sp in soup.find('div', class_ = 'cb-col cb-col-33 text-black').find_all('div', class_ = 'cb-col cb-col-60 cb-lst-itm-sm')[:3]]
      details += basic_info

      height = soup.find('div', class_ = 'cb-col cb-col-33 text-black').find('div', class_ = 'cb-col cb-col-60').text.strip()
      details.append(height)

      ranks = [sp.text.strip() for sp in soup.find('div', class_ = 'cb-col cb-col-33 text-black').find_all('div', class_ = 'cb-col cb-col-25 cb-plyr-rank text-right')]
      details += ranks

      details.append(link)    

      # Scraping Table Data
      bat_table = soup.find_all('table', class_ = 'table cb-col-100 cb-plyr-thead')[0]
      bow_table = soup.find_all('table', class_ = 'table cb-col-100 cb-plyr-thead')[1]

      df_bat = create_df(bat_table)
      df_bow = create_df(bow_table)

      player_id = link.split('/')[-1]

      bat_path = 'Scores/bat_' + player_id + '.csv'
      bow_path = 'Scores/bow_' + player_id + '.csv'

      df_bat.to_csv(bat_path, index = False)
      df_bow.to_csv(bow_path, index = False)

      details.append(bat_path)
      details.append(bow_path)
      
      final_data.append(details)

  except:
      details = []


df = pd.DataFrame(final_data, columns = ['name','team','date_of_birth','palce_of_birth','role','height',
                                         'test_bat_rank','odi_bat_rank','t20_bat_rank',
                                         'test_bow_rank','odi_bow_rank','t20_bow_rank',
                                         'link','bat_path','bow_path'])

df.to_csv('cricket.csv', index = False)

100%|██████████| 5736/5736 [1:32:43<00:00,  1.03it/s]


In [ ]:
df

,name,team,date_of_birth,palce_of_birth,role,height,test_bat_rank,odi_bat_rank,t20_bat_rank,test_bow_rank,odi_bow_rank,t20_bow_rank,link,bat_path,bow_path
0,Sachin Tendulkar,India,"Apr 24, 1973 (49 years)","Bombay (now Mumbai), Maharashtra",Batsman,5 ft 5 in,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/25,Scores/bat_25.csv,Scores/bow_25.csv
1,Virender Sehwag,India,"Oct 20, 1978 (44 years)",Delhi,Batsman,5 ft 8 in (173 cm),--,--,--,--,--,--,https://www.cricbuzz.com/profiles/26,Scores/bat_26.csv,Scores/bow_26.csv
2,Rahul Dravid,India,"Jan 11, 1973 (49 years)","Indore, Madhya Pradesh",Batsman,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/27,Scores/bat_27.csv,Scores/bow_27.csv
3,VVS Laxman,India,"Nov 01, 1974 (48 years)","Hyderabad, Andhra Pradesh",--,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/28,Scores/bat_28.csv,Scores/bow_28.csv
4,Sourav Ganguly,India,"Jul 08, 1972 (50 years)","Calcutta (now Kolkata), Bengal",Batsman,5 ft 11 in,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29,Scores/bat_29.csv,Scores/bow_29.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5712,Imran Khan,Bahrain,"Apr 01, 1990 (32 years)",--,Bowler,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29552,Scores/bat_29552.csv,Scores/bow_29552.csv
5713,Shahrukh Quddus,Kuwait,"Dec 11, 1996 (26 years)",--,Bowler,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29669,Scores/bat_29669.csv,Scores/bow_29669.csv
5714,Ateeq Iqbal,Hong Kong,"Dec 10, 1991 (31 years)",--,Bowler,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29711,Scores/bat_29711.csv,Scores/bow_29711.csv
5715,Ali Zaheer,Kuwait,"Feb 08, 1994 (28 years)",Kuwait,Batsman,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29718,Scores/bat_29718.csv,Scores/bow_29718.csv


In [ ]:
def get_cols(file_path, sep):  
    cols = []
    col_names = pd.read_csv(file_path).columns
    for _ in ['Test','ODI','T20I','IPL']:
        for col_name in col_names[1:]:
            cols.append(sep + "_" + _ + "_" + col_name)
    return cols

def get_data(file_path, seq_len):
    
    df_ = pd.read_csv(file_path)

    test = ['-' for i in range(seq_len)]
    odi  = ['-' for i in range(seq_len)]
    t20  = ['-' for i in range(seq_len)]
    ipl  = ['-' for i in range(seq_len)]

    for i in df_.values:   
        if i[0] == 'Test':
            test = i[1:]
        if i[0] == 'ODI':
            odi = i[1:]
        if i[0] == 'T20I':
            t20 = i[1:]
        if i[0] == 'IPL':
            ipl = i[1:]

        final =  [_ for _ in test]
        final += [_ for _ in odi ]
        final += [_ for _ in t20 ]
        final += [_ for _ in ipl ]

    return final

In [ ]:
bat_data = []
bow_data = []

for i in df.values:
    bat_path = i[-2]
    bat_data.append(get_data(bat_path,13))
    
    bow_path = i[-1]
    bow_data.append(get_data(bow_path,12))

In [ ]:
df_bat = pd.DataFrame(bat_data, columns = get_cols('Scores/bat_1413.csv', 'BT'))
df_bat.to_csv('bat.csv', index = False)

df_bow = pd.DataFrame(bow_data, columns = get_cols('Scores/bow_1413.csv', 'BW'))
df_bow.to_csv('ball.csv', index = False)

In [ ]:
df_fin = pd.concat((df,df_bat,df_bow), axis = 1)
df_fin.to_csv('players.csv', index = False)

In [ ]:
df_fin

,name,team,date_of_birth,palce_of_birth,role,height,test_bat_rank,odi_bat_rank,t20_bat_rank,test_bow_rank,...,BW_IPL_B,BW_IPL_Runs,BW_IPL_Wkts,BW_IPL_BBI,BW_IPL_BBM,BW_IPL_Econ,BW_IPL_Avg,BW_IPL_SR,BW_IPL_5W,BW_IPL_10W
0,Sachin Tendulkar,India,"Apr 24, 1973 (49 years)","Bombay (now Mumbai), Maharashtra",Batsman,5 ft 5 in,--,--,--,--,...,36,58,0,0/7,0/7,9.67,0.0,0.0,0,0
1,Virender Sehwag,India,"Oct 20, 1978 (44 years)",Delhi,Batsman,5 ft 8 in (173 cm),--,--,--,--,...,136,235,6,2/18,2/18,10.37,39.17,22.67,0,0
2,Rahul Dravid,India,"Jan 11, 1973 (49 years)","Indore, Madhya Pradesh",Batsman,--,--,--,--,--,...,-,-,-,-,-,-,-,-,-,-
3,VVS Laxman,India,"Nov 01, 1974 (48 years)","Hyderabad, Andhra Pradesh",--,--,--,--,--,--,...,-,-,-,-,-,-,-,-,-,-
4,Sourav Ganguly,India,"Jul 08, 1972 (50 years)","Calcutta (now Kolkata), Bengal",Batsman,5 ft 11 in,--,--,--,--,...,276,363,10,2/21,2/21,7.89,36.3,27.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5712,Imran Khan,Bahrain,"Apr 01, 1990 (32 years)",--,Bowler,--,--,--,--,--,...,-,-,-,-,-,-,-,-,-,-
5713,Shahrukh Quddus,Kuwait,"Dec 11, 1996 (26 years)",--,Bowler,--,--,--,--,--,...,-,-,-,-,-,-,-,-,-,-
5714,Ateeq Iqbal,Hong Kong,"Dec 10, 1991 (31 years)",--,Bowler,--,--,--,--,--,...,-,-,-,-,-,-,-,-,-,-
5715,Ali Zaheer,Kuwait,"Feb 08, 1994 (28 years)",Kuwait,Batsman,--,--,--,--,--,...,-,-,-,-,-,-,-,-,-,-


## Data Analysis

In [ ]:
df

,name,team,date_of_birth,palce_of_birth,role,height,test_bat_rank,odi_bat_rank,t20_bat_rank,test_bow_rank,odi_bow_rank,t20_bow_rank,link,bat_path,bow_path
0,Sachin Tendulkar,India,"Apr 24, 1973 (49 years)","Bombay (now Mumbai), Maharashtra",Batsman,5 ft 5 in,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/25,Scores/bat_25.csv,Scores/bow_25.csv
1,Virender Sehwag,India,"Oct 20, 1978 (44 years)",Delhi,Batsman,5 ft 8 in (173 cm),--,--,--,--,--,--,https://www.cricbuzz.com/profiles/26,Scores/bat_26.csv,Scores/bow_26.csv
2,Rahul Dravid,India,"Jan 11, 1973 (49 years)","Indore, Madhya Pradesh",Batsman,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/27,Scores/bat_27.csv,Scores/bow_27.csv
3,VVS Laxman,India,"Nov 01, 1974 (48 years)","Hyderabad, Andhra Pradesh",--,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/28,Scores/bat_28.csv,Scores/bow_28.csv
4,Sourav Ganguly,India,"Jul 08, 1972 (50 years)","Calcutta (now Kolkata), Bengal",Batsman,5 ft 11 in,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29,Scores/bat_29.csv,Scores/bow_29.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5712,Imran Khan,Bahrain,"Apr 01, 1990 (32 years)",--,Bowler,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29552,Scores/bat_29552.csv,Scores/bow_29552.csv
5713,Shahrukh Quddus,Kuwait,"Dec 11, 1996 (26 years)",--,Bowler,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29669,Scores/bat_29669.csv,Scores/bow_29669.csv
5714,Ateeq Iqbal,Hong Kong,"Dec 10, 1991 (31 years)",--,Bowler,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29711,Scores/bat_29711.csv,Scores/bow_29711.csv
5715,Ali Zaheer,Kuwait,"Feb 08, 1994 (28 years)",Kuwait,Batsman,--,--,--,--,--,--,--,https://www.cricbuzz.com/profiles/29718,Scores/bat_29718.csv,Scores/bow_29718.csv
